In [2]:
from os import listdir, walk
from os.path import isfile, join
import pandas as pd
import pandas_market_calendars as mcal

ModuleNotFoundError: No module named 'pandas'

In [2]:
path = '../../../../Data/forex/oanda/minute/'
currencies = ['audcad/'] #, 'audchf/']

In [3]:
for currency in currencies:
    files = [f for f in listdir(path + currency) if 2020 <= int(f.split('_')[0][:4]) <= 2022]

In [4]:
files[:5]

['20200102_quote.zip',
 '20200103_quote.zip',
 '20200105_quote.zip',
 '20200106_quote.zip',
 '20200107_quote.zip']

In [5]:
import datetime
def convert_to_time(t):
    t /= 1000
    seconds = int(t%60)
    t /= 60
    minutes = int(t%60)
    t /= 60
    hours = int(t%24)
    return datetime.time(hours, minutes, seconds)

In [6]:
# symbols = ["AUD", "CAD", "CHF", "EUR", "GBP", "NZD", "USD", "JPY"]
markets = {mcal.get_calendar('EUREX'): ['EUR', 'CHF', 'GBP'], mcal.get_calendar('HKEX'): ['JPY', 'AUD', 'NZD'], mcal.get_calendar('XMEX'): ['CAD', 'USD']}
def market_open(symbol, sch, d):
    symbol = symbol[:3].upper()
    market = None
    for k, v in markets.items():
         if symbol in v:
             market = k
    return cal.open_at_time(sch, d)

AttributeError: module 'pandas_market_calendars' has no attribute 'get_calendar'

In [31]:
import talib

CANDLESTICKS = talib.__function_groups__['Pattern Recognition']
INDICATORS = {
    'BBANDS': {'method': 'BBANDS', 'period': 20}, 'SMA_SLOW': {'method': 'SMA', 'period': 200},
    'SMA_FAST': {'method': 'SMA', 'period': 50}, 'EMA_SLOW': {'method': 'EMA', 'period': 20},
    'EMA_FAST': {'method': 'EMA', 'period': 9}, 'ATR': {'method': 'ATR', 'period': 14},
    'RSI': {'method': 'RSI', 'period': 14}
}


def get_indicators(df):
    df = df.copy()
    opens = df['open'].values
    highs = df['high'].values
    lows = df['low'].values
    closes = df['close'].values
    # Add indicator columns to DataFrame
    for indicator_name, indicator_values in INDICATORS.items():
        if indicator_name == 'BBANDS':
            ub, _, lb = getattr(talib, indicator_values['method'])(closes, indicator_values['period'])
            df[f'{indicator_name}_UB'], df[f'{indicator_name}_LB'] = ub, lb
        elif indicator_name == 'ATR':
            df[indicator_name] = getattr(talib, indicator_values['method'])(highs,lows,closes,indicator_values['period'])
        else:
            df[indicator_name] = getattr(talib, indicator_values['method'])(closes, indicator_values['period'])
    return df


def get_candlestick_markers(df):
    df = df.copy()
    for method in CANDLESTICKS:
        df[method] = getattr(talib, method)(df["open"].values, df["high"].values, df["low"].values,
                                              df["close"].values)
    return df

In [196]:
train = pd.DataFrame()
test = pd.DataFrame()
sizes = []
symbol = currencies[0]
for i, f in enumerate(files):
    if (i+1) % 100 == 0:
        print(i)
    date = f.split('_')[0]
    temp = pd.read_csv(path+currency+f, names=['time', 'open_bid', 'high_bid', 'low_bid', 'close_bid', 'volume', 'open_ask', 'high_ask', 'low_ask', 'close_ask', 'volume2'])
    sizes.append(temp.shape[0])
    if temp.shape[0] < 1400:
        continue
    del temp['volume']
    del temp['volume2']
    temp['date'] = date
    temp['time'] = temp['time'].apply(convert_to_time)
    temp['datetime'] = temp['time'].apply(lambda x: datetime.datetime(int(date[:4]), int(date[4:6]), int(date[6:]), x.hour, x.minute, x.second, tzinfo=datetime.timezone.utc))
    
    symbol = symbol[:3].upper()
    market = None
    for k, v in markets.items():
         if symbol in v:
             market = k
    if market is None:
        continue
    sch = market.schedule(temp['datetime'].iloc[0].date(), temp['datetime'].iloc[0].date())
    temp['market_open'] = temp['datetime'].apply(lambda d: market.open_at_time(sch, d))
    temp = temp[temp['market_open'] == True]
    del temp['market_open']
    temp['spread'] = temp['close_ask'] - temp['close_bid']
    for c in temp.columns:
        if c.endswith('ask'):
            temp[c[:-4]+'_1min'] = temp[c]
            del temp[c]
        elif c.endswith('bid'):
            del temp[c]
#     temp = get_indicators(temp)
#     temp = get_candlestick_markers(temp)
    if f.startswith('2020'):
        train = pd.concat([train, temp])
    else:
        test = pd.concat([test, temp])

99
199
299


In [197]:
for c in train.columns:
    if c.endswith('ask'):
        train[c[:-4]+'_1min'] = train[c]
for c in test.columns:
    if c.endswith('ask'):
        test[c[:-4]+'_1min'] = test[c]

In [198]:
train.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)

In [199]:
len(train), len(test)

(65007, 14520)

In [200]:
train

,time,date,spread,open_1min,high_1min,low_1min,close_1min
datetime,,,,,,,
2020-01-02 01:30:00+00:00,01:30:00,20200102,0.00024,0.91060,0.91066,0.91053,0.91063
2020-01-02 01:31:00+00:00,01:31:00,20200102,0.00026,0.91065,0.91078,0.91062,0.91073
2020-01-02 01:32:00+00:00,01:32:00,20200102,0.00023,0.91075,0.91077,0.91064,0.91072
2020-01-02 01:33:00+00:00,01:33:00,20200102,0.00028,0.91072,0.91077,0.91063,0.91065
2020-01-02 01:34:00+00:00,01:34:00,20200102,0.00027,0.91064,0.91064,0.91058,0.91059
...,...,...,...,...,...,...,...
2020-12-30 07:55:00+00:00,07:55:00,20201230,0.00027,0.97922,0.97930,0.97916,0.97922
2020-12-30 07:56:00+00:00,07:56:00,20201230,0.00031,0.97922,0.97933,0.97921,0.97929
2020-12-30 07:57:00+00:00,07:57:00,20201230,0.00031,0.97929,0.97930,0.97919,0.97926


In [201]:
def cumsample(x):
    if x.shape[0] == 0:
        return pd.DataFrame()
    else:
        return pd.DataFrame({'open': [x['open_1min'].iloc[0]]*len(x), 'close': x['close_1min'], 'high': x['close_1min'].cummax(), 'low': x['close_1min'].cummin()})

In [202]:
%%time
timeframes = ['5min', '15min', '30min', '60min']
dfs = pd.DataFrame()
for frame in timeframes:
    print(frame)
    df = train.resample(frame).apply(cumsample)
    df.index = df.index.droplevel()
    df.columns = [c+'_'+frame for c in df.columns]
    dfs = pd.concat([dfs, df], axis=1)
train = pd.concat([train, dfs], axis=1)

5min
15min
30min
60min


In [205]:
%%time
timeframes = ['5min', '15min', '30min', '60min']
dfs = pd.DataFrame()
for frame in timeframes:
    print(frame)
    df = test.resample(frame).apply(cumsample)
    df.index = df.index.droplevel()
    df.columns = [c+'_'+frame for c in df.columns]
    dfs = pd.concat([dfs, df], axis=1)
test = pd.concat([test, dfs], axis=1)

5min
15min
30min
60min
CPU times: user 26 s, sys: 330 ms, total: 26.4 s
Wall time: 26.1 s


In [204]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 65007 entries, 2020-01-02 01:30:00+00:00 to 2020-12-30 07:59:00+00:00
Data columns (total 23 columns):
time           65007 non-null object
date           65007 non-null object
spread         65007 non-null float64
open_1min      65007 non-null float64
high_1min      65007 non-null float64
low_1min       65007 non-null float64
close_1min     65007 non-null float64
open_5min      65007 non-null float64
close_5min     65007 non-null float64
high_5min      65007 non-null float64
low_5min       65007 non-null float64
open_15min     65007 non-null float64
close_15min    65007 non-null float64
high_15min     65007 non-null float64
low_15min      65007 non-null float64
open_30min     65007 non-null float64
close_30min    65007 non-null float64
high_30min     65007 non-null float64
low_30min      65007 non-null float64
open_60min     65007 non-null float64
close_60min    65007 non-null float64
high_60min     65007 non-null float64
low_60min 

In [34]:
timeframes = ['5min', '15min', '30min', '60min']
train_timeframes = {}
test_timeframes = {}
for frame in timeframes:
    train_timeframes[frame] = train.resample(frame, on='datetime').agg({'open_bid': 'first', 'open_ask': 'first', 
                                                      'close_bid': 'first', 'close_ask': 'last',
                                                      'high_bid': 'first', 'high_ask': 'max', 
                                                      'low_bid': 'first', 'low_ask': 'min'})
for frame in timeframes:
    test_timeframes[frame] = test.resample(frame, on='datetime').agg({'open_bid': 'first', 'open_ask': 'first', 
                                                      'close_bid': 'first', 'close_ask': 'last',
                                                      'high_bid': 'first', 'high_ask': 'max', 
                                                      'low_bid': 'first', 'low_ask': 'min'})

In [36]:
train_timeframes['5min'].join?

Signature: join(a, *p)
Docstring:
Join two or more pathname components, inserting '/' as needed.
If any component is an absolute path, all previous path components
will be discarded.  An empty last part will result in a path that
ends with a separator.
File:      /opt/miniconda3/lib/python3.6/posixpath.py
Type:      function


In [ ]:
train_timeframes['5min'].join

In [ ]:
timeframes = ['5min', '15min', '30min', '60min']
train_timeframes = {t: pd.DataFrame() for t in timeframes}
test_timeframes = {t: pd.DataFrame() for t in timeframes}
for d_set, df in zip(['train', 'test'], [train, test]):
    df_frame = pd.DataFrame()
    for time, sample in df.resample(teimframes[-1], on='datetime'):
        if sample.shape[0] == 0:
            continue
        for frame in timeframes[:-1]:
        temp = pd.DataFrame([], index=[time])
        for t in ['bid', 'ask']:
            temp[f'open_{t}_{timeframe}'] = sample.iloc[0][f'open_{t}']
            temp[f'close_{t}'] = sample.iloc[-1][f'close_{t}']
            temp[f'high_{t}'] = sample[f'high_{t}'].max()
            temp[f'low_{t}'] = sample[f'low_{t}'].min()
        df_frame = df_frame.append(temp)
    if d_set == 'train':
        train_timeframes = pd.concat([train_timeframes, df_frame], axis=1)
    else:
        test_timeframes = pd.concat([test_timeframes, df_frame], axis=1)

In [ ]:
%%time
timeframes = ['5min', '15min', '30min', '1hour']
train_timeframes = pd.DataFrame()
test_timeframes = pd.DataFrame()
for d_set, df in zip(['train', 'test'], [train, test]):
    for timeframe in timeframes:
        print(timeframe)
        df_frame = pd.DataFrame()
        for time, sample in df.resample('5min', on='datetime'):
            if sample.shape[0] == 0:
                continue
            temp = pd.DataFrame([], index=[time])
            for t in ['bid', 'ask']:
                temp[f'open_{t}_{timeframe}'] = sample.iloc[0][f'open_{t}']
                temp[f'close_{t}'] = sample.iloc[-1][f'close_{t}']
                temp[f'high_{t}'] = sample[f'high_{t}'].max()
                temp[f'low_{t}'] = sample[f'low_{t}'].min()
            df_frame = df_frame.append(temp)
        if d_set == 'train':
            train_timeframes = pd.concat([train_timeframes, df_frame], axis=1)
        else:
            test_timeframes = pd.concat([test_timeframes, df_frame], axis=1)

5min


In [ ]:
train_timeframes.info()

In [209]:
train.to_csv(f"data/train/5_15_30_1h_timeframes.csv")
test.to_csv(f"data/test/5_15_30_1h_timeframes.csv")

In [17]:
for episode in episodes_train:
    episode.to_csv(f"data/train/{episode['date'].iloc[0]}", index=False)
for episode in episodes_test:
    episode.to_csv(f"data/test/{episode['date'].iloc[0]}", index=False)

In [207]:
train.columns

Index(['time', 'date', 'spread', 'open_1min', 'high_1min', 'low_1min',
       'close_1min', 'open_5min', 'close_5min', 'high_5min', 'low_5min',
       'open_15min', 'close_15min', 'high_15min', 'low_15min', 'open_30min',
       'close_30min', 'high_30min', 'low_30min', 'open_60min', 'close_60min',
       'high_60min', 'low_60min'],
      dtype='object')

In [208]:
train['close'] = train['close_1min']
test['close'] = test['close_1min']

In [210]:
train

,time,date,spread,open_1min,high_1min,low_1min,close_1min,open_5min,close_5min,high_5min,...,low_15min,open_30min,close_30min,high_30min,low_30min,open_60min,close_60min,high_60min,low_60min,close
datetime,,,,,,,,,,,,,,,,,,,,,
2020-01-02 01:30:00+00:00,01:30:00,20200102,0.00024,0.91060,0.91066,0.91053,0.91063,0.91060,0.91063,0.91063,...,0.91063,0.9106,0.91063,0.91063,0.91063,0.91060,0.91063,0.91063,0.91063,0.91063
2020-01-02 01:31:00+00:00,01:31:00,20200102,0.00026,0.91065,0.91078,0.91062,0.91073,0.91060,0.91073,0.91073,...,0.91063,0.9106,0.91073,0.91073,0.91063,0.91060,0.91073,0.91073,0.91063,0.91073
2020-01-02 01:32:00+00:00,01:32:00,20200102,0.00023,0.91075,0.91077,0.91064,0.91072,0.91060,0.91072,0.91073,...,0.91063,0.9106,0.91072,0.91073,0.91063,0.91060,0.91072,0.91073,0.91063,0.91072
2020-01-02 01:33:00+00:00,01:33:00,20200102,0.00028,0.91072,0.91077,0.91063,0.91065,0.91060,0.91065,0.91073,...,0.91063,0.9106,0.91065,0.91073,0.91063,0.91060,0.91065,0.91073,0.91063,0.91065
2020-01-02 01:34:00+00:00,01:34:00,20200102,0.00027,0.91064,0.91064,0.91058,0.91059,0.91060,0.91059,0.91073,...,0.91059,0.9106,0.91059,0.91073,0.91059,0.91060,0.91059,0.91073,0.91059,0.91059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 07:55:00+00:00,07:55:00,20201230,0.00027,0.97922,0.97930,0.97916,0.97922,0.97922,0.97922,0.97922,...,0.97889,0.9797,0.97922,0.97976,0.97887,0.97915,0.97922,0.97976,0.97887,0.97922
2020-12-30 07:56:00+00:00,07:56:00,20201230,0.00031,0.97922,0.97933,0.97921,0.97929,0.97922,0.97929,0.97929,...,0.97889,0.9797,0.97929,0.97976,0.97887,0.97915,0.97929,0.97976,0.97887,0.97929
2020-12-30 07:57:00+00:00,07:57:00,20201230,0.00031,0.97929,0.97930,0.97919,0.97926,0.97922,0.97926,0.97929,...,0.97889,0.9797,0.97926,0.97976,0.97887,0.97915,0.97926,0.97976,0.97887,0.97926


In [211]:
((0.91065 - 0.91073) / 0.91073 + 1) * (1 - 0.005) ** 2 - 1

-0.010061965401381379

In [212]:
(1 - 0.005) ** 2

0.990025